In [1]:
import json
import requests
import sqlite3

In [2]:
coordinates = "55.7263,37.6503,55.7316,37.6864"
text_query = f""" [out: json];
    nwr({coordinates});
    out geom; """

In [3]:
overpass_url = "https://overpass.openstreetmap.ru/api/interpreter"
response = requests.get(overpass_url, params={'data': text_query})
json_data = response.json()
del json_data["version"]
del json_data["generator"]
del json_data["osm3s"]
with open("output_data.json", "w") as outfile:
    json.dump(json_data, outfile, indent=4)

In [4]:
nodes = []
node_tags = []
ways = []
way_tags = []
way_nodes = []
relations = []
rel_tags = []
members = []
for i in range(len(json_data["elements"])):
    if(json_data["elements"][i]["type"] == "node"):
        node_id = json_data["elements"][i]["id"]
        lat     = json_data["elements"][i]["lat"]
        lon     = json_data["elements"][i]["lon"]
        nodes.append((node_id, lat, lon))
        if('tags' in json_data['elements'][i].keys()):
            for k in json_data['elements'][i]['tags'].keys():
                tag_key = k
                tag_val = json_data['elements'][i]['tags'][k]
                node_tags.append((node_id, tag_key, tag_val))
    if(json_data["elements"][i]["type"] == "way"):
        way_id = json_data["elements"][i]["id"]
        ways.append((way_id,))
        if('tags' in json_data['elements'][i].keys()):
            for k in json_data['elements'][i]['tags'].keys():
                tag_key = k
                tag_val = json_data['elements'][i]['tags'][k]
                way_tags.append((way_id, tag_key, tag_val))
        if('nodes' in json_data['elements'][i].keys()):
            for j in range(len(json_data['elements'][i]['nodes'])):
                node_id = json_data['elements'][i]['nodes'][j]
                way_nodes.append((way_id, node_id, j))
    if(json_data["elements"][i]["type"] == "relation"):
        rel_id = json_data["elements"][i]["id"]
        relations.append((rel_id,))
        if('tags' in json_data['elements'][i].keys()):
            for k in json_data['elements'][i]['tags'].keys():
                tag_key = k
                tag_val = json_data['elements'][i]['tags'][k]
                rel_tags.append((rel_id, tag_key, tag_val))
        if('members' in json_data['elements'][i].keys()):
            for j in range(len(json_data['elements'][i]['members'])):
                member_type = json_data['elements'][i]['members'][j]['type']
                member_id = json_data['elements'][i]['members'][j]['ref']
                member_role = json_data['elements'][i]['members'][j]['role']
                members.append((rel_id, member_type, member_id, member_role, j))
        


In [5]:
with open('tests/nodes_test.txt', 'w') as file:
    for x in nodes:
        print(x, file=file)
file.close()

In [6]:
with open('tests/node_tags_test.txt', 'w') as file:
    for x in node_tags:
        print(x, file=file)
file.close()

In [7]:
with open('tests/ways_test.txt', 'w') as file:
    for x in ways:
        print(x, file=file)
file.close()

In [8]:
with open('tests/way_tags_test.txt', 'w') as file:
    for x in way_tags:
        print(x, file=file)
file.close()

In [9]:
with open('tests/way_nodes_test.txt', 'w') as file:
    for x in way_nodes:
        print(x, file=file)
file.close()

In [10]:
with open('tests/relations_test.txt', 'w') as file:
    for x in relations:
        print(x, file=file)
file.close()

In [11]:
with open('tests/rel_tags_test.txt', 'w') as file:
    for x in rel_tags:
        print(x, file=file)
file.close()

In [12]:
with open('tests/members_test.txt', 'w') as file:
    for x in members:
        print(x, file=file)
file.close()

In [13]:
print('nodes', len(nodes))
print('node_tags', len(node_tags))
print('ways', len(ways))
print('way_tags', len(way_tags))
print('way_nodes', len(way_nodes))
print('relations', len(relations))
print('rel_tags', len(rel_tags))
print('members', len(members))

nodes 5120
node_tags 1949
ways 1086
way_tags 4064
way_nodes 8810
relations 168
rel_tags 864
members 5928


In [16]:
con = sqlite3.connect('shadow.db')
cur = con.cursor()

cur.executemany("INSERT INTO nodes VALUES (?, ?, ?)", nodes)
cur.executemany("INSERT INTO node_tags VALUES (?, ?, ?)", node_tags)
cur.executemany("INSERT INTO ways VALUES (?)", ways)
cur.executemany("INSERT INTO way_tags VALUES (?, ?, ?)", way_tags)
cur.executemany("INSERT INTO way_nodes VALUES (?, ?, ?)", way_nodes)
cur.executemany("INSERT INTO relations VALUES (?)", relations)
cur.executemany("INSERT INTO rel_tags VALUES (?, ?, ?)", rel_tags)
cur.executemany("INSERT INTO rel_members VALUES (?, ?, ?, ?, ?)", members)


con.commit()

In [17]:
cur.close()
con.close()